In [1]:
import pickle
import re

demodir = '/home/anton/Documents/Tulane/Research/ImageNomer/data/anton/cohorts/UKB/'

demo = pickle.load(open(f'{demodir}/demographics.pkl', 'rb'))

print(list(demo.keys()))

['f_22001_0_0', 'f_21003_3_0', 'f_21003_2_0', 'f_21000_0_0', 'CaAfCarDiff', 'CaIndDiff', 'CaChiDiff', 'f_21000_0_0_comb']


In [49]:
import pickle

field = 25753

basedir = '/run/media/anton/AntonData1/UKB_MRI_data_11092022/'

fc2 = pickle.load(open(f'{basedir}/{field}_2.pkl', 'rb'))
fc3 = pickle.load(open(f'{basedir}/{field}_3.pkl', 'rb'))

print(len(fc2), len(fc3))

40471 2896


In [60]:
import numpy as np

first = []
second = []

for sub in demo['f_21003_3_0']:
    if sub not in demo['f_21003_2_0']:
        continue
    try:
        p0 = np.load(f'{demodir}/fc/{sub}_task-20227-2_fc.npy')
        p1 = np.load(f'{demodir}/fc/{sub}_task-20227-3_fc.npy')
#         p0 = fc2[sub]
#         p1 = fc3[sub]
        first.append(p0)
        second.append(p1)
    except:
        continue
    
first = np.stack(first)
second = np.stack(second)

print(len(first), len(second))

2722 2722


In [64]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

N = 400

errs = []
nullerrs = []

for _ in range(10):
    idcs = np.random.permutation(len(first))

    a = first-second
    b = second-first

    x = np.concatenate([a[idcs[:N]], b[idcs[N:N*2]]])
    y = np.concatenate([np.zeros(N), np.ones(N)])

    xtr, xt, ytr, yt = train_test_split(x, y, stratify=y, train_size=0.5)

    reg = LogisticRegression(max_iter=1000).fit(xtr, ytr)
    yhat = reg.predict(xt)
    err = np.mean(yhat == yt)
    nullerr = np.mean(yt)
    if nullerr < 0.5:
        nullerr = 1-nullerr
    print(err)
    print(nullerr)
    errs.append(err)
    nullerrs.append(nullerr)
    
print(np.mean(errs), np.std(errs))
print(np.mean(nullerrs), np.std(nullerrs))

0.7025
0.5
0.73
0.5
0.7575
0.5
0.7625
0.5
0.7425
0.5
0.7325
0.5
0.77
0.5
0.7725
0.5
0.76
0.5
0.7275
0.5
0.74575 0.021361472327533968
0.5 0.0
